# [SC57 - Working with big, multi-dimensional geoscientific datasets in Python: a tutorial introduction to xarray](http://meetingorganizer.copernicus.org/EGU2017/session/25651)  
  
  
Original notebook by [Stephan Hoyer](http://stephanhoyer.com), Rossbypalooza, 2016.  
Modified by Edward Byers, Matthew Gidden and [Fabien Maussion](http://fabienmaussion.info/) for EGU General Assembly 2017, Vienna, Austria
Modified by C. Gentemann for GHRSST Science Team Tutorial 2019, Rome, Italy
Modified by C.Gentemann for future tutorials  
    
  
  
**Convenors**
* [Dr Chelle Gentemann](mailto:gentemann@esr.org)    - Earth and Space Research, USA
* [Dr Marisol Garcia-Reyes](mailto:marisolgr@faralloninstitute.org)  - Farallon Institute, USA 
-------------


# Structure of this tutorial

1. Opening data
1. Collocating satellite data with a cruise dataset



# 1. Key features of `xarray`

-------------------

## Import python packages

You are going to want numpy, pandas, matplotlib.pyplot and xarray

In [ ]:
import warnings
warnings.simplefilter('ignore') # filter some warning messages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs

from pathlib import Path
import configparser
from lxml import objectify
#for search capabilites import podaacpy
import podaac.podaac as podaac
from podaac import drive as podaacdrive
import podaac.podaac_utils as putil
# then create an instance of the Podaac class
p = podaac.Podaac()
with open('./podaac.ini', 'r') as f:
    config = configparser.ConfigParser()
    config.read_file(f)
    d = podaacdrive.Drive(None, 
                          config['drive']['urs_username'], 
                          config['drive']['urs_password'])

## A nice cartopy tutorial is [here](http://earthpy.org/tag/visualization.html)

# Collocating Saildrone cruise data with MUR SSTs 

* read in the Saildrone data 

In [ ]:
#if offline use first url
#url = './data/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc'
url = 'https://podaac-opendap.jpl.nasa.gov/opendap/hyrax/allData/insitu/L2/saildrone/Baja/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc'
ds_usv = 

## The NCEI trajectory format uses 'obs' as the coordinate.  This is an example of an 'older' style of data formatting that doesn't really mesh well with modern software capabilities. 

* So, let's change that by using [.swap_dims](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.swap_dims.html) to change the coordinate from `obs` to `time`
* Another thing, `latitude` and `longitude` are just long and annoying, lets [.rename](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.rename.html) them to `lat` and `lon`

* Finally, the first and last part of the cruise the USV is being towed, so let's only include data from `2018-04-12T02` to `2018-06-10T18`


In [ ]:
ds_usv2 = ds_usv.isel(trajectory=0)

ds_usv2 = ds_usv2.swap_dims({'obs':'time'})

ds_usv2 = ds_usv2.rename({'longitude':'lon','latitude':'lat'})

ds_usv_subset = ds_usv2.sel(time=slice('2018-04-12T02','2018-06-10T18')) 

start_time=pd.to_datetime(str(ds_usv2.time.min().data)).strftime('%Y-%m-%dT%H:%m:%SZ') 
end_time=pd.to_datetime(str(ds_usv2.time.max().data)).strftime('%Y-%m-%dT%H:%m:%SZ') 

print('start: ',start_time,'end: ',end_time)

### Let's open 2 months of 0.2 km AVHRR OI SST data

`xarray`can open multiple files at once using string pattern matching.  
  
  In this case we open all the files that match our `filestr`, i.e. all the files for the 2080s. 
  
  Each of these files (compressed) is approximately 800 MB.

In [ ]:
dataset_id = 'PODAAC-GHCMC-4FM03'  #CMC SST looked up on podaac website
#dataset_id = 'PODAAC-GHGMR-4FJ04'  #MUR SST looked up on podaac website

gresult = p.granule_search(dataset_id=dataset_id,
                           start_time=start_time,
                           end_time=end_time,
                           items_per_page='100')

urls = putil.PodaacUtils.mine_opendap_urls_from_granule_search(gresult)

urls = [w[:-5] for w in urls]  #remove html from urls

# Let's see what one day looks like
`xr.open_dataset(urls[0])`

In [ ]:
ds_sst = 

# Plot a day using cartopy

In [ ]:
ax = plt.axes(projection=ccrs.Orthographic(-80, 35))  #this sets projection for plot

ds_sst.analysed_sst.plot(ax=ax, transform=ccrs.PlateCarree())  #this tells projection of data

#ax.coastlines(); 

# mask out land values

`ds_sst.where(ds_sst.mask<=1)`

In [ ]:
ds_sst_masked = 


# Change the colormap, colorscale, and add land

`cmap='jet'`
`vmin=-1,vmax=34`
`ax.stock_img();`

## Now open multiple files (lazy) using [.open_mfdataset](http://xarray.pydata.org/en/stable/generated/xarray.open_mfdataset.html#xarray.open_mfdataset)

* use the option `coords = 'minimal'`

In [ ]:
ds_sst = xr.open_mfdataset('./data/avhrr_oi/*.nc',coords='minimal').isel(zlev=0)
ds_sst

## Again with the 0-360 vs -180-180.  Change it up below!

* `ds_sst.coords['lon'] = (np.mod(ds_sst.coords['lon'] + 180,360) - 180)`

* `ds_sst = ds_sst.sortby(ds_sst.lon)`




In [ ]:
ds_sst.coords['lon'] = (np.mod(ds_sst.coords['lon'] + 180,360) - 180)
ds_sst = ds_sst.sortby(ds_sst.lon)
ds_sst

`xarray` even puts them in the right order for you.

In [ ]:
ds_sst.time

How big is all this data uncompressed? Will it fit into memory?
Use `.nbytes` / 1e9  to convert it into gigabytes

In [ ]:
print('size: :', ds_sst.nbytes / 1e6)

# Xarray interpolation won't run on chunked dimensions.  
1. First let's subset the data to make it smaller to deal with by using the cruise lat/lons
    * Find the max/min of the lat/lon using `.lon.min().data`

1. Now load the data into memory (de-Dask-ify) it using `.load()`  


In [ ]:
#Step 1 from above
lon_min,lon_max = ds_usv_subset.lon.min().data,ds_usv_subset.lon.max().data
lat_min,lat_max = ds_usv_subset.lat.min().data,ds_usv_subset.lat.max().data

subset = ds_sst.sel(lon=slice(lon_min,lon_max),
                    lat=slice(lat_min,lat_max))



In [ ]:
print('size: :', subset.nbytes / 1e6)

In [ ]:
#Step 2 from above
subset.load()

# Collocate USV data with MUR data
There are different options when you interpolate.  First, let's just do a linear interpolation using [.interp()](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.interp.html#xarray.Dataset.interp)

`Dataset.interp(coords=None, method='linear', assume_sorted=False, kwargs={}, **coords_kwargs))`



In [ ]:
ds_collocated = subset.interp(lat=ds_usv_subset.lat,
                              lon=ds_usv_subset.lon,
                              time=ds_usv_subset.time,
                              method='linear')

# Collocate USV data with MUR data
There are different options when you interpolate.  First, let's just do a nearest point rather than interpolate the data
`method = 'nearest'`

In [ ]:
ds_collocated_nearest = subset.interp(lat=ds_usv_subset.lat,
                                      lon=ds_usv_subset.lon,
                                      time=ds_usv_subset.time,
                                      method='nearest')

## Now, calculate the different in SSTs and print the [.mean()](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.mean.html#xarray.DataArray.mean) and [.std()](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.std.html#xarray.DataArray.std)
For the satellite data we need to use `sst` and for the USV data we need to use `TEMP_CTD_MEAN`

In [ ]:
dif = ds_collocated_nearest.sst-ds_usv_subset.TEMP_CTD_MEAN

print('mean difference = ',dif.mean().data)
print('STD = ',dif.std().data)

# xarray can do more!

* concatentaion
* open network located files with openDAP
* import and export Pandas DataFrames
* .nc dump to 
* groupby_bins
* resampling and reduction

For more details, read this blog post: http://continuum.io/blog/xray-dask


In [ ]:
#ds_collocated_nearest.to_netcdf('./data/new file.nc')

## Where can I find more info?

### For more information about xarray

- Read the [online documentation](http://xarray.pydata.org/)
- Ask questions on [StackOverflow](http://stackoverflow.com/questions/tagged/python-xarray)
- View the source code and file bug reports on [GitHub](http://github.com/pydata/xarray/)

### For more doing data analysis with Python:

- Thomas Wiecki, [A modern guide to getting started with Data Science and Python](http://twiecki.github.io/blog/2014/11/18/python-for-data-science/)
- Wes McKinney, [Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do) (book)

### Packages building on xarray for the geophysical sciences

For analyzing GCM output:

- [xgcm](https://github.com/xgcm/xgcm) by Ryan Abernathey
- [oogcm](https://github.com/lesommer/oocgcm) by Julien Le Sommer
- [MPAS xarray](https://github.com/pwolfram/mpas_xarray) by Phil Wolfram
- [marc_analysis](https://github.com/darothen/marc_analysis) by Daniel Rothenberg

Other tools:

- [windspharm](https://github.com/ajdawson/windspharm): wind spherical harmonics by Andrew Dawson
- [eofs](https://github.com/ajdawson/eofs): empirical orthogonal functions by Andrew Dawson
- [infinite-diff](https://github.com/spencerahill/infinite-diff) by Spencer Hill 
- [aospy](https://github.com/spencerahill/aospy) by Spencer Hill and Spencer Clark
- [regionmask](https://github.com/mathause/regionmask) by Mathias Hauser
- [salem](https://github.com/fmaussion/salem) by Fabien Maussion

Resources for teaching and learning xarray in geosciences:
- [Fabien's teaching repo](https://github.com/fmaussion/teaching): courses that combine teaching climatology and xarray
